In [45]:
from __future__ import absolute_import, division, print_function, unicode_literals

%load_ext autoreload
%autoreload 2

# import numpy as np
# import scipy as sp
# import nibabel as nib
# from nibabel.testing import data_path
# import matplotlib.pyplot as plt
# import matplotlib as mpl
# import pandas as pd
%matplotlib inline
%config InlineBackend.figure_format = 'retina'  # adapt plots for retina displays

# from IPython.core.debugger import set_trace
# import torchvision
# from mpl_toolkits.mplot3d import Axes3D
import os
import utils
# from tqdm import tqdm_notebook
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# import multiprocessing
from torch.utils.tensorboard import SummaryWriter
# import json
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torchvision import datasets, transforms, models
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
# import datasets, models
# import torchsample
import torchio as tio
# import skimage.transform as skTrans
import wandb
wandb.init(project="cnn-interpretability")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [46]:
image_dir = "/raid/users/alfin/01_P_Classification_all"

In [47]:
# For Two classes:

Labels = []
filenames = []
NotFitImages = []
for root, dirs, files in os.walk(os.path.abspath(image_dir)):
    for file in files:
        if file.split(".")[0].split("_")[-1] == "Normal":
            Labels.append(1)
            filenames.append(os.path.join(root, file))
        elif file.split(".")[0].split("_")[-1] == "Dementia":
            Labels.append(0)
            filenames.append(os.path.join(root, file))
        else:
            NotFitImages.append(file)

print(len(filenames))


251


Data split

In [48]:
from sklearn.model_selection import train_test_split
labels = torch.nn.functional.one_hot(torch.as_tensor(Labels)).float()
train_filenames, val_filenames, train_labels, val_labels = train_test_split(filenames, labels)

Preprocessing
(Normalization and Skull Stripping)


In [49]:
# New Dataset class to read all images.
# TODO: Maybe move this up and make it general across all experiments.

class UNOMICDataset(Dataset):
    
    def __init__(self, filenames, labels, mask=None, remove_nan=True, transform=None):
            
        self.filenames = filenames
        self.labels = labels
        self.mask = mask
        self.remove_nan = remove_nan
        self.transform = transform
        
        # Required by torchsample.
        self.num_inputs = 1
        self.num_targets = 1
        
        self.mean = 1
        self.std = 1
        
    def __len__(self):
        return len(self.filenames)
        # return len(self.result1)
    
    def __getitem__(self, idx):

        label = self.labels[idx]
        # print(self.labels.shape)
        struct_arr = utils.load_nifti(self.filenames[idx], mask=self.mask, remove_nan=self.remove_nan)
        
        # TODO: Maybe do this (at least the normalization) via transforms.
        # TODO: Maybe do normalization via mean as well.
        struct_arr = struct_arr / (self.std)
        struct_arr = struct_arr[None]  # add (empty) channel dimension
        struct_arr = torch.from_numpy(struct_arr).float()
        # print(label)
        # print(label)
        return struct_arr, label


In [50]:
# train_dataset = UNOMICDataset(train_filenames, train_labels)
# val_dataset = UNOMICDataset(val_filenames, val_labels)
# transform = transforms.Compose([transforms.ToPILImage(), transforms.Resize((91, 109, 91)), transforms.ToTensor()])

train_dataset = UNOMICDataset(train_filenames, train_labels)
val_dataset = UNOMICDataset(val_filenames, val_labels)

In [51]:
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True, num_workers=1, pin_memory=torch.cuda.is_available()) 
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False, num_workers=1, pin_memory=torch.cuda.is_available())


# train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=1, shuffle=True, num_workers=1) 
# val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=1, shuffle=False, num_workers=1)



Create Model


Model-1

In [52]:
# Original model from Fabi.
class ClassificationModel3D(nn.Module):
    def __init__(self):
        nn.Module.__init__(self)
        self.Conv_1 = nn.Conv3d(1, 64, 3)
        self.Conv_2 = nn.Conv3d(64, 64, 3)
        self.Conv_3 = nn.Conv3d(64, 64, 3)
        self.Conv_4 = nn.Conv3d(64, 64, 3)
        self.dense_1 = nn.Linear(128, 2)
        self.elu = nn.ELU()
        
    def forward(self, inp):
        x = self.elu(self.Conv_1(inp))
        x = F.max_pool3d(x, 3)
        x = self.elu(self.Conv_2(x))
        x = F.max_pool3d(x, 3)
        x = self.elu(self.Conv_3(x))
        x = self.elu(self.Conv_4(x))
        x = F.max_pool3d(x, 3)
        x = x.view(x.size(0), -1)
        x = self.dense_1(x)

        # print(x.shape)
        
        # Note that no sigmoid is applied here, because the network is used in combination with BCEWithLogitsLoss,
        # which applies sigmoid and BCELoss at the same time to make it numerically stable.
        
        return x

In [53]:
net = ClassificationModel3D()

In [54]:
if torch.cuda.is_available():
    net = net.cuda()
    cuda_device = torch.cuda.current_device()
    print('Moved network to GPU, cuda_device: {}'.format(cuda_device))
else:
    cuda_device = -1
    print('GPU not available, cuda_device: {}'.format(cuda_device))


Moved network to GPU, cuda_device: 0


In [58]:
import warnings
warnings.filterwarnings('ignore')
model = net
model.cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_func = torch.nn.MSELoss() 
train_data_loader = train_loader
test_data_loader = val_loader
trainImagesLen = len(train_filenames)
testImagesLen = len(val_filenames)

accuracy = 0
count = 0
# Training and Testing
total = 0
writer = SummaryWriter()
EPOCHS = 100
for epoch in range(1,EPOCHS):        
    for step, (x, y) in enumerate(train_data_loader):
        b_x = Variable(x).cuda()   # batch x (image)
        b_y = Variable(y).cuda()   # batch y (target) Target size (torch.Size([1, 2])) 
        output = model(b_x)[0]          
        # print('actual: {} and predicted: {}'.format(y, output))
        # break
        loss = loss_func(torch.t(output), b_y)   
        optimizer.zero_grad()           
        loss.backward()                 
        optimizer.step()

        loss, current = loss.item(), step * len(x)
        wandb.log({ 'loss': loss})
        print(f"loss: {loss:>7f} , Epoch: {epoch}, Iterations: [{current:>5d}/{trainImagesLen:>5d}]")
        # writer.add_scalar("train_loss", loss(), step * epoch + step)
    count += 1
    if count % 2 == 0:
        correct = 0
        total = 0
        for x1, y1 in test_data_loader:

            test_x = Variable(x1).cuda()
            test_y = Variable(y1).cuda()
            test_output = model(test_x)
            predicted = torch.max(test_output.data, 1)[1].data.squeeze()

            total += len(test_y)
            correct += (predicted == test_y).sum()

        accuracy = 100 * correct / float(total)
        
        wandb.log({'accuracy': accuracy})
    # average_accuracy = accuracy / accuracy_count
        print(f"accuracy: {accuracy:>2f} after Epoch {step}")
    print("-"*15)
        


loss: 0.336523 , Epoch: 1, Iterations: [    0/  188]
loss: 5.696758 , Epoch: 1, Iterations: [    1/  188]
loss: 0.446174 , Epoch: 1, Iterations: [    2/  188]
loss: 0.347896 , Epoch: 1, Iterations: [    3/  188]
loss: 1.037198 , Epoch: 1, Iterations: [    4/  188]
loss: 2.004324 , Epoch: 1, Iterations: [    5/  188]
loss: 0.432092 , Epoch: 1, Iterations: [    6/  188]
loss: 0.807398 , Epoch: 1, Iterations: [    7/  188]
loss: 0.250626 , Epoch: 1, Iterations: [    8/  188]
loss: 0.450468 , Epoch: 1, Iterations: [    9/  188]
loss: 4.716986 , Epoch: 1, Iterations: [   10/  188]
loss: 0.203750 , Epoch: 1, Iterations: [   11/  188]
loss: 0.802405 , Epoch: 1, Iterations: [   12/  188]
loss: 0.583233 , Epoch: 1, Iterations: [   13/  188]
loss: 0.321136 , Epoch: 1, Iterations: [   14/  188]
loss: 0.045300 , Epoch: 1, Iterations: [   15/  188]
loss: 0.300669 , Epoch: 1, Iterations: [   16/  188]
loss: 0.086510 , Epoch: 1, Iterations: [   17/  188]
loss: 0.095535 , Epoch: 1, Iterations: [   18/

Train-1

In [ ]:
model = net
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_func = torch.nn.MSELoss() 
train_data_loader = train_loader
test_data_loader = val_loader
trainImagesLen = len(train_filenames)
# Training and Testing
writer = SummaryWriter()
EPOCHS = 100
for epoch in range(EPOCHS):        
    for step, (x, y) in enumerate(train_data_loader):
        b_x = Variable(x)   # batch x (image)
        b_y = Variable(y)   # batch y (target) Target size (torch.Size([1, 2])) 
        output = model(b_x)[0]          
        # print('actual: {} and predicted: {}'.format(y, output))
        # break
        loss = loss_func(torch.t(output), b_y)   
        optimizer.zero_grad()           
        loss.backward()                 
        optimizer.step()

        loss, current = loss.item(), step * len(x)
        print(f"loss: {loss:>7f} , Epoch: {epoch}, Iterations: [{current:>5d}/{trainImagesLen:>5d}]")
        wandb.log({ 'loss': loss})

        # writer.add_scalar("train_loss", loss(), step * epoch + step)
        # if step % 2 == 0:
        #     test_x = Variable(test_data_loader)
        #     test_output, last_layer = model(test_x)
        #     pred_y = torch.max(test_output, 1)[1].data.squeeze()
        #     accuracy = sum(pred_y == test_y) / float(test_y.size(0))
        #     print('Epoch: ', epoch, '| train loss: %.4f' % loss.data[0], '| test accuracy: %.2f' % accuracy)

/home/alfin/miniconda3/lib/python3.9/site-packages/nilearn/image/resampling.py:531: UserWarning: Casting data from int16 to float32
  warnings.warn("Casting data from %s to %s" % (data.dtype.name, aux))


loss: 0.012943 , Epoch: 0, Iterations: [    0/  188]
loss: 0.000232 , Epoch: 0, Iterations: [    1/  188]
loss: 0.007772 , Epoch: 0, Iterations: [    2/  188]
loss: 0.006826 , Epoch: 0, Iterations: [    3/  188]
loss: 0.001721 , Epoch: 0, Iterations: [    4/  188]
loss: 0.000086 , Epoch: 0, Iterations: [    5/  188]
loss: 0.002432 , Epoch: 0, Iterations: [    6/  188]
loss: 0.004210 , Epoch: 0, Iterations: [    7/  188]


KeyboardInterrupt: 